In [1]:
from pyspark.sql import SparkSession


In [2]:
spark = SparkSession.builder.appName('cluster').getOrCreate()
df = spark.read.csv('Mall_Customers.csv', inferSchema=True, header=True)
df.printSchema()

root
 |-- CustomerID: integer (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Annual Income (k$): integer (nullable = true)
 |-- Spending Score (1-100): integer (nullable = true)

